In [1]:
import numpy as np 
import pandas as pd 
import nltk
import re
import string
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from subprocess import check_output
from nltk.stem import WordNetLemmatizer
import sys  

reload(sys)  
sys.setdefaultencoding('utf8')

In [2]:
print(check_output(["ls", "test.csv"]).decode("utf8"))
test = pd.read_csv("./test.csv")

In [3]:
punctations = string.punctuation
stopWrds = stopwords.words('english')
def clean_title(data):
    title = data.title
    title = title.apply(lambda x : x.lower())
    return title

    
testTitle = clean_title(test)

In [4]:
testTitle[1]

'what is your simplest explanation of the string theory?'

In [5]:
test.content[0]

'<p>I often hear about subatomic particles having a property called "spin" but also that it doesn\'t actually relate to spinning about an axis like you would think. Which particles have spin? What does spin mean if not an actual spinning motion?</p>\n'

In [6]:
def clean_content(data):
    data.content = [x.encode('ascii', errors='ignore') for x in data.content]
   # data.content = data.content.encode('ascii', errors='ignore')
    content = data.content
    content = content.apply(lambda x: x.lower())
    content = content.apply(lambda x: re.sub(r'\<[^<>]*\>','',x))
    content = content.apply(lambda x: re.sub(r'\n','',x))
    return content
testContent = clean_content(test)

In [7]:
testContent[0]

'i often hear about subatomic particles having a property called "spin" but also that it doesn\'t actually relate to spinning about an axis like you would think. which particles have spin? what does spin mean if not an actual spinning motion?'

In [29]:
#using rake 
stoppath = "stoplist.txt"
import RAKE as rake
stopWrds = stopwords.words('english')
len(stopWrds)
rake_object = rake.Rake("stoplist.txt")

In [30]:
text = "Natural language processing (NLP) deals with the application of computational models to text or speech data. Application areas within NLP include automatic (machine) translation between languages; dialogue systems, which allow a human to interact with a machine using natural language; and information extraction, where the goal is to transform unstructured text into structured (database) representations that can be searched and browsed in flexible ways. NLP technologies are having a dramatic impact on the way people interact with computers, on the way people interact with each other through the use of language, and on the way people access the vast amount of linguistic data now in electronic form. From a scientific viewpoint, NLP involves fundamental questions of how to structure formal models (for example statistical models) of natural language phenomena, and of how to design algorithms that implement these models. In this course you will study mathematical and computational models of language, and the application of these models to key problems in natural language processing. The course has a focus on machine learning methods, which are widely used in modern NLP systems: we will cover formalisms such as hidden Markov models, probabilistic context-free grammars, log-linear models, and statistical models for machine translation. The curriculum closely follows a course currently taught by Professor Collins at Columbia University, and previously taught at MIT."

In [31]:
keywords = rake_object.run(testContent[0])

In [32]:
keywords

[('actual spinning motion', 8.0),
 ('property called', 4.0),
 ('subatomic particles', 3.5),
 ('spinning', 2.0),
 ('particles', 1.5),
 ('relate', 1.0),
 ('doesn', 1.0),
 ('hear', 1.0),
 ('spin', 1.0),
 ('axis', 1.0)]

In [130]:
keywordsList = []
for i in range(0,len(testContent)):
    keywords = rake_object.run(testTitle[i])
    keywordsList.append(keywords)

In [131]:
keywordsList[0:2]
keywords[0][0]

'gravity manipulation'

In [132]:
keywordsList  = [[i[0] for i in x] for x in keywordsList ] 
keywordsList[0]

['subatomic particles', 'relates', 'spin']

In [133]:
keywordsList = [[i for i in x if len(i.split(" "))<3 ] for x in keywordsList ]

In [134]:
len(keywordsList)

81926

In [135]:
keywordsPosTags  = [nltk.pos_tag(x) for x in keywordsList]
keywordsPosTags[0]



[('subatomic particles', 'NNS'), ('relates', 'VBZ'), ('spin', 'NNS')]

In [137]:
tags2 = []
for taglist in keywordsPosTags:
    goodterm = [i[0].encode('utf-8') for i in taglist if i[1][0] in "NN"]
    tags2.append(goodterm)

In [142]:
tags2

[['subatomic particles', 'spin'],
 ['simplest explanation', 'string theory'],
 ['lie theory', 'particle physics', 'representations'],
 ['determinism'],
 ['hamilton', 'principle'],
 ['sound'],
 ['experiment'],
 ['sky', 'blue', 'night', 'sunrise/set', 'day'],
 ['energy'],
 ['monte carlo'],
 ['banking', 'bicycle'],
 ['electromagnetic field', 'velocity'],
 ['quantum mechanics', 'interaction', 'difference', 'measurement'],
 [],
 ['lay explanation', 'relativity'],
 ['coriolis effect', 'sink/bathtub', 'whirl/vortex', 'show'],
 ['energy', 'repel', 'magnets'],
 ['real world', 'correspondence', 'equations'],
 ['impressions', 'mathematics'],
 [],
 [],
 [],
 ['change colour', 'polarised materials', 'stress'],
 ['intuitive explanation', 'gouy phase'],
 ['cancer treatment', 'proton therapy'],
 ['yang-baxter equation', 'physicists', 'solutions'],
 ['mnemonics', 'materials', 'properties'],
 ['neutrons repel'],
 ['interaction'],
 [],
 ['proton'],
 ['general relativity', 'gravity consistent', 'newton', 

In [143]:
tags3 = []
for taglist in tags2:
    goodterm = [i for i in taglist if len(i) > 2]
    tags3.append(goodterm)

In [165]:
tags4 = []
for taglist in tags3:
    goodterm = []
    for i in taglist:
      #  if len(i.split(" ")) > 1:
      #      goodterm.append(i.split(" ")[0] + "-"+i.split(" ")[1])
      #  else:
        goodterm.append(i)
    tags4.append(goodterm)

In [166]:
tags4[0:5]

[['subatomic particles', 'spin'],
 ['simplest explanation', 'string theory'],
 ['lie theory', 'particle physics', 'representations'],
 ['determinism'],
 ['hamilton', 'principle']]

In [167]:
topTags = ["homework-and-exercises","quantum-mechanics","newtonian-mechanics"]
tags5 = []
for taglist in tags4:
    if(len(taglist) == 0):
        tags5.append(topTags)
    else:
        tags5.append(taglist)


In [168]:
title_corpus = [" ".join(terms)  for terms in tags5]

sub_title = pd.DataFrame(
    {
        'id': test.id,
        'tags': title_corpus
    
  
    })

sub_title.to_csv('mysub_titleRAKE.csv',index=False)